<a href="https://colab.research.google.com/github/khorneflakes-dev/Proyecto-Final-YELP/blob/main/review_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [6]:
df_business_id = pd.read_csv('/content/drive/MyDrive/output/business_index.csv')
df_business_id.columns = ['id_business', 'business_id']
df_user_id = pd.read_parquet('/content/drive/MyDrive/codes_users.parquet.gzip')

In [7]:
path = '/content/drive/MyDrive/test/Dataset Yelp/review.json'

In [8]:
main_chunk = pd.read_json(path, lines=True, chunksize=1000000)

funcion para pasar al chunk

In [9]:
def review_etl(df):
  df.drop(['text'], axis=1, inplace=True)
  df_aux = pd.merge(df, df_business_id, left_on='business_id', right_on='business_id', how='inner')
  df_aux.drop(['business_id'], axis=1, inplace=True)
  df_aux['id_review'] = df_aux.index
  df_aux.drop(['review_id'], axis=1, inplace=True)

  df_aux2 = pd.merge(df_aux, df_user_id, left_on='user_id', right_on='user_id', how='inner')
  df_aux2.drop(['user_id'], axis=1, inplace=True)
  df_aux2.drop(['useful','funny','cool'], axis=1, inplace=True)
  df_aux2 = df_aux2.rename(columns={"n_user_id": "id_user"})
  return df_aux2

In [10]:
chunk_list = []

In [11]:
for chunk in main_chunk:
  chunk_list.append(review_etl(chunk))
  print('hi')

hi
hi
hi
hi
hi
hi
hi


In [12]:
len(chunk_list)

7

In [13]:
df_final = pd.concat(chunk_list)

In [14]:
df_final = df_final[['id_review','id_user','id_business','stars','date']]

In [15]:
df_final['date']= pd.to_datetime(df_final['date'])
df_final['year'] = df_final['date'].dt.year

In [16]:
df_final['month'] = df_final['date'].dt.month

In [17]:
df_final.drop(['date'], axis=1, inplace=True)

In [25]:
df_final['id_review'] = list(range(1, len(df_final)+1, 1))

In [26]:
df_review_inicial = df_final[df_final['year'] < 2022]
df_review_incremental = df_final[df_final['year']==2022]

In [27]:
df_review_incremental.shape

(26257, 6)

In [28]:
df_review_inicial.shape

(6021799, 6)

In [32]:
df_review_inicial.to_parquet('/content/drive/MyDrive/output/reviews_inicial.parquet.gzip', compression='gzip')
df_review_incremental.to_parquet('/content/drive/MyDrive/output/reviews_incremental.parquet.gzip', compression='gzip')

In [30]:
len(df_final.id_review.drop_duplicates())

6048056

In [31]:
len(df_final)

6048056